<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Python: Projeto Final
Caderno de **Exercicios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)\
Aluno [Natanael Quintiliano](https://www.linkedin.com/in/natanaelquintiliano/)

---

# Topicos  <a name="topicos"></a>
1. [Apresentando o problema](#apresentando)
2. [As bibliotecas que usaremos](#bibliotecas)
3. [Exploração de dados](#explorando)
4. [Transformação e limpeza de dados](#tranformando)
5. [Análise de dados](#analisando)
6. [Insights](#insights)
7. [Conclusões](#conclusoes)

**Caso o codigo não rode aqui no Kaggle basta baixar o notebook e rodar no Google Colab.**

---

# 1. Apresentando o problema <a name="apresentando"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topicos)
     


Vamos explorar dados de crédito presentes neste neste [link](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv). Os dados estão no formato CSV e contém informações sobre clientes de uma instituição financeira. Em especial, estamos interessados em explicar a segunda coluna, chamada de **default**, que indica se um cliente é adimplente(`default = 0`), ou inadimplente (`default = 1`), ou seja, queremos entender o porque um cliente deixa de honrar com suas dívidas baseado no comportamento de outros atributos, como salário, escolaridade e movimentação financeira. Uma descrição completa dos atributos está abaixo.

> O atributo de interesse (`default`) é conhecido como **variável resposta** ou **variável dependente**, já os demais atributos que buscam explicá-la (`idade`, `salário`, etc.) são conhecidas como **variáveis explicatívas**, **variáveis independentes** ou até **variáveis preditoras**.



| Coluna  | Descrição |
| ------- | --------- |
| id      | Número da conta |
| default | Indica se o cliente é adimplente (0) ou inadimplente (1) |
| idade   | --- |
| sexo    | --- |
| depedentes | --- |
| escolaridade | --- |
| estado_civil | --- |
| salario_anual | Faixa do salario mensal multiplicado por 12 |
| tipo_cartao | Categoria do cartao: blue, silver, gold e platinium |
| meses_de_relacionamento | Quantidade de meses desde a abertura da conta |
| qtd_produtos | Quantidade de produtos contratados |
| iteracoes_12m | Quantidade de iteracoes com o cliente no último ano |
| meses_inatico_12m | Quantidade de meses que o cliente ficou inativo no último ano |
| limite_credito | Valor do limite do cartão de crédito |
| valor_transacoes_12m | Soma total do valor das transações no cartão de crédito no último ano |
| qtd_transacoes_12m | Quantidade total de transações no cartão de crédito no último ano |



# 2. As biliotecas que usaremos <a name="bibliotecas"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topicos)

Para ler os dados:

In [ ]:
import pandas as pd

Para visualizar os dados:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

O esquema de cores que usaremos em nossos graficos:


In [ ]:
sns.set_style("whitegrid")

# 3. Exploração de dados <a name="explorando"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topicos)

Vamos começar lendos os dados num dataframe:

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv', na_values='na')

Conferindo o dataframe:

In [ ]:
df.head(n=10)

Com o dados em mãos, vamos conhecer um pouco melhor a estrutura do nosso conjunto de dados.

### **3.1. Estrutura**

In [ ]:
df.shape # retorna uma tupla (qtd linhas, qtd colunas)

In [ ]:
df[df['default'] == 0].shape

In [ ]:
df[df['default'] == 1].shape

In [ ]:
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape

In [ ]:
print(f"A proporcão clientes adimplentes é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A proporcão clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")

### **3.2. Schema**

In [ ]:
df.head(n=5)

Colunas e seus respectivos tipos de dados:

In [ ]:
df.dtypes


Atributos **categóricos**.

In [ ]:
df.select_dtypes('object').describe().transpose()

Atributos **numéricos**.

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

### **3.3. Dados faltantes**

Podemos verificar quais colunas possuem dados faltantes.

In [ ]:
df.isna().any()

A função abaixo levanta algumas estatisticas sobre as colunas dos dados faltantes.

In [ ]:
def stats_dados_faltantes(df: pd.DataFrame) -> None:

  stats_dados_faltantes = []
  for col in df.columns:
    if df[col].isna().any():
      qtd, _ = df[df[col].isna()].shape
      total, _ = df.shape
      dict_dados_faltantes = {col: {'quantidade': qtd, "porcentagem": round(100 * qtd/total, 2)}}
      stats_dados_faltantes.append(dict_dados_faltantes)

  for stat in stats_dados_faltantes:
    print(stat)

In [ ]:
stats_dados_faltantes(df=df)

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 0])

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 1])

# 4. Transformação e limpeza de dados <a name="tranformando"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topicos)

Agora que conhecemos melhor a natureza do nosso conjunto de dados, vamos conduzir uma atividade conhecida como *data wrangling* que consiste na transformação e limpeza dos dados do conjunto para que possam ser melhor analisados. Em especial, vamos remover:

 - Corrigir o *schema* das nossas colunas;
 - Remover os dados faltantes.

### **4.1. Correção de schema**

Na etapa de exploração, notamos que as colunas **limite_credito** e **valor_transacoes_12m** estavam sendo interpretadas como colunas categóricas (`dtype = object`).

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].dtypes

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].head(n=5)

Vamos criar uma função `lambda` para limpar os dados. Mas antes, vamos testar sua aplicação através do método funcional `map`:

In [ ]:
fn = lambda valor: float(valor.replace(".", "").replace(",", "."))

valores_originais = ['12.691,51', '8.256,96', '3.418,56', '3.313,03', '4.716,22']
valores_limpos = list(map(fn, valores_originais))

print(valores_originais)
print(valores_limpos)

Com a função `lambda` de limpeza pronta, basta aplica-la nas colunas de interesse.

In [ ]:
df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(fn)
df['limite_credito'] = df['limite_credito'].apply(fn)

Vamos descrever novamente o *schema*:

In [ ]:
df.dtypes

 Atributos **categóricos**.

In [ ]:
df.select_dtypes('object').describe().transpose()

Atributos **numéricos**.

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

### **4.2. Remoção de dados faltantes**

Como o pandas está ciente do que é um dados faltante, a remoção das linhas problemáticas é trivial.

In [ ]:
df.dropna(inplace=True)

Vamos analisar a estrutura dos dados novamente.

In [ ]:
df.shape

In [ ]:
df[df['default'] == 0].shape

In [ ]:
df[df['default'] == 1].shape

In [ ]:
qtd_total_novo, _ = df.shape
qtd_adimplentes_novo, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novo, _ = df[df['default'] == 1].shape

In [ ]:
print(f"A proporcão adimplentes ativos é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A nova proporcão de clientes adimplentes é de {round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)}%")
print("")
print(f"A proporcão clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")
print(f"A nova proporcão de clientes inadimplentes é de {round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)}%")

# 5. Analise de dados <a name="analisando"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topicos)

Os dados estão prontos, vamos criar diversas visualizações para correlacionar variáveis explicativas com a variável resposta para buscar entender qual fator leva um cliente a inadimplencia. E para isso, vamos sempre comparar a base com todos os clientes com a base de adimplentes e inadimplentes.

In [ ]:
df_adimplente = df[df['default'] == 0]

In [ ]:
df_inadimplente = df[df['default'] == 1]

### **5.1. Visualizações categóricas**

Nesta seção, vamos visualizar a relação entre a variável resposta **default** com os atributos categóricos.

In [ ]:
df.select_dtypes('object').head(n=5)

Função para vermos a relação entre variáveis categóricas.

In [ ]:
def relacao_variaveis_categoricas(df: pd.DataFrame,coluna:str,titulos:list) -> None:
  eixo = 0
  max_y = 0
  max = df.select_dtypes('object').describe()[coluna]['freq'] * 1.1

  figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

  for dataframe in [df, df_adimplente, df_inadimplente]:

    df_to_plot = dataframe[coluna].value_counts().to_frame()
    df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace=True)
    df_to_plot[coluna] = df_to_plot.index
    df_to_plot.sort_values(by=[coluna], inplace=True)
    df_to_plot.sort_values(by=[coluna])

    f = sns.barplot(x=df_to_plot[coluna], y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
    f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(0, max_y))

    eixo += 1

  return figura.show()


#### **5.1.1. Escolaridade**

In [ ]:
""" Este comando roda apenas no google Colab estou tentando descobrir porque, alguma possivel solução deixe nos comentarios, grato.
relacao_variaveis_categoricas(
    df = df
    ,coluna =  'escolaridade'
    ,titulos = ['Escolaridade dos Clientes', 'Escolaridade dos Clientes Adimplentes', 'Escolaridade dos Clientes Inadimplentes']
)
"""

#### **5.1.2. Salário anual**

In [ ]:
""" Este comando roda apenas no google Colab estou tentando descobrir porque, alguma possivel solução deixe nos comentarios, grato.
relacao_variaveis_categoricas(
    df=df
    ,coluna = 'salario_anual'
    ,titulos = ['Salário Anual dos Clientes', 'Salário Anual dos Clientes Adimplentes', 'Salário Anual dos Clientes Inadimplentes']
)
"""

### **5.2. Visualizações numéricas**

Nesta seção, vamos visualizar a relação entre a variável resposta **default** com os atributos numéricos.

In [ ]:
df.drop(['id', 'default'], axis=1).select_dtypes('number').head(n=5)

Função para vermos a relação entre variaveis categóricas.

In [ ]:
def relacao_variaveis_numericas(df: pd.DataFrame,coluna:str,titulos:list) -> None:
  eixo = 0
  max_y = 0
  figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

  for dataframe in [df, df_adimplente, df_inadimplente]:

    f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(0, max_y))

    eixo += 1

  figura.show()

#### **5.2.1. Quantidade de transações nos últimos 12 meses**

In [ ]:
relacao_variaveis_numericas(
    df=df
    ,coluna = 'qtd_transacoes_12m'
    ,titulos = ['Qtd. de Transações no Último Ano', 'Qtd. de Transações no Último Ano de Adimplentes', 'Qtd. de Transações no Último Ano de Inadimplentes']
)

#### **5.2.2. Valor das transações nos últimos 12 meses**

In [ ]:
relacao_variaveis_numericas(
    df=df
    ,coluna = 'valor_transacoes_12m'
    ,titulos = ['Valor das Transações no Último Ano', 'Valor das Transações no Último Ano de Adimplentes', 'Valor das Transações no Último Ano de Inadimplentes']
)

#### **5.2.3 - Cruzamento entre: valor de transações nos últimos 12 meses x quantidade de transações nos últimos 12 meses.**

In [ ]:
f = sns.relplot(x='valor_transacoes_12m', y='qtd_transacoes_12m', data=df, hue='default')
_ = f.set(
    title='Relação entre Valor e Quantidade de Transações no Último Ano',
    xlabel='Valor das Transações no Último Ano',
    ylabel='Quantidade das Transações no Último Ano'
  )

#### **5.2.4. Limite de crédito**

In [ ]:
relacao_variaveis_numericas(
    df=df
    ,coluna = 'limite_credito'
    ,titulos = ['Limite de Credito', 'Limite de Credito de Adimplentes', 'Limite de Credito de Inadimplentes']
)

#### **5.2.5. Idade**

In [ ]:
relacao_variaveis_numericas(
    df=df
    ,coluna = 'idade'
    ,titulos = ['Idade', 'Idade de Adimplentes', 'Idade de Inadimplentes']
)

# 6. Insights <a name="insights"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topicos)

Após analisarmos e observamos graficamente quais variáveis mais poderiam impactar na variável ser ou não inadiplente, obersavmos que duas variáveis se destacaram `valor_transacoes_12m` e `qtd_transacoes_12m`. A seguir olharemos percentualmente o impacto das mesmas em ser ou não inadiplente.

#### **6.1. Valor transações nos ultimos 12 meses**

Analisando o grafico começamos a estimar qual range de valores tem mais clientes inadiplentes.

In [ ]:
# Clientes que transacionaram menos de $3000,00
df_transacoes_vmaximo = df[df['valor_transacoes_12m']<3000]

In [ ]:
# Quantidade de clientes que transacionaram menos de $3000,00 e são inadiplentes
df_transac_inadimplente,_ = df_transacoes_vmaximo[df_transacoes_vmaximo['default'] == 1].shape

In [ ]:
## Quantidade de clientes que transacionaram menos de $3000,00
df_transacoes_vmaximo_qtd,_ = df[df['valor_transacoes_12m']<3000].shape

Vericamos que o range de clientes que tivereram transações abaixo de $3000,00 são os que porcentualmente tem mais clientes inadiplentes, 30,89% dos clientes desse range são `default = 1` quase o dobro da porcentagem de 15,72% inadiplentes em relação ao total de clientes.

In [ ]:
# Porcentagem dos clientes que transacionaram menos de $3000,00 e são inadiplentes
round(100 * df_transac_inadimplente / df_transacoes_vmaximo_qtd, 2)

Abaixo veremos que os clientes totais desse range são 39,68% dos clientes, parece pouco? Mas...





In [ ]:
#Porcentagem dos total de clientes que transacionaram menos de $3000,00
round(100 * df_transacoes_vmaximo_qtd / qtd_total_novo, 2)

... os clientes inadiplentes desse range representam aproximadente 78% dos clientes inadiplentes.

In [ ]:
#Porcentagem dos clientes que transacionaram menos de $3000,00 e são inadiplentes em relação ao total de inadiplentes
round(100 * df_transac_inadimplente/qtd_inadimplentes_novo, 2)

E somente os clientes inadiplentes desse range representam  12,26% dos total de clientes, adiplentes ou não.

In [ ]:
#Porcentagem dos clientes que transacionaram menos de $3000,00 e são inadiplentes em relação ao total de clientes
round(100 * df_transac_inadimplente/qtd_total_novo, 2)

Em resumo qualquer mudança na inadiplencia desses clientes impacta e muito no indice geral.


#### **6.2. Quantidade de transações nos ultimos 12 meses**

Analisando o grafico começamos a estimar qual range de valores tem mais clientes inadiplentes.

In [ ]:
df_transacoes_qtdmaximo =  df[df['qtd_transacoes_12m']<60]

In [ ]:
#Quantidade de clientes que transacionaram menos de 60 vezes e são inadiplentes
df_transac_qtdinadimplente,_ = df_transacoes_qtdmaximo[df_transacoes_qtdmaximo['default'] == 1].shape

In [ ]:
#Quantidade de clientes que transacionaram menos de 60 vezes
df_transacoes_qtdmaximo_qtd,_ = df[df['qtd_transacoes_12m']<60].shape

Vericamos que o range de clientes que tivereram menos de 60 transações são os que porcentualmente tem mais clientes inadiplentes, 33,12% dos clientes desse range são default = 1 quase o dobro da porcentagem de 15,72% inadiplentes em relação ao total de clientes.

In [ ]:
# Porcentagem dos clientes que transacionaram menos de 60 vezes e são inadiplentes
round(100 * df_transac_qtdinadimplente / df_transacoes_qtdmaximo_qtd, 2)

Abaixo veremos que os clientes totais desse range são 39,99% dos clientes, tanbém parece pouco? Mas assim como na variáveis anterior...

In [ ]:
#Porcentagem dos clientes totais que transacionaram menos de 60 vezes
round(100 * df_transacoes_qtdmaximo_qtd /qtd_total_novo, 2)

... os clientes inadiplentes desse range representam a maioria (84,28%) dos clientes inadiplentes.

In [ ]:
#Porcentagem de clientes que transacionaram menos de 60 vezes e sao inadiplentes em relacao ao total de inadiplentes
round(100 * df_transac_qtdinadimplente/qtd_inadimplentes_novo, 2)

E novamente somente os clientes inadiplentes desse range representam acima 12% dos total de clientes, adiplentes ou não. Relembrando que o total de inadiplentes é proximo de 16%.

In [ ]:
#Porcentagem de clientes que transacionaram menos de 60 vezes e sao inadiplentes em relacao ao total de clientes
round(100 * df_transac_qtdinadimplente/qtd_total_novo, 2)

#### 6.3\. Cruzando variáveis

Agora vamos cruzar essa duas variáveis para verificarmos a porcentagem de clientes inadiplentes que transacioaram menos de $3000,00 e transacionaram menos de 60 vezes no ultimo ano.




Chegamos ao valor expressivo de 92,54%, dos clientes que trasacionaram menos de $3000 também transacionaram menos de 60 vezes no ultimo ano.

In [ ]:
#Porcentagem dos clientes inadiplente que transacioaram menos de $3000 e transacionaram menos de 60 vezes
round(100 * df_transac_inadimplente/df_transac_qtdinadimplente, 2)

Apos verificarmos que o cruzamento dessas variaves, vamos verificar a porcentagem dos clientes inadiplementes que possuem ambas condições: trasancionaram menos de 60 vezes e trasancionaram menos de $3000 no ultimo ano.

In [ ]:
df_transac_qtdinad = df_transacoes_qtdmaximo[df_transacoes_qtdmaximo['default'] == 1]
df_inadimplente_cruzado = df_transac_qtdinad[df_transac_qtdinad['valor_transacoes_12m']<3000]


In [ ]:
df_qtdinadimplente_cruzado,_ = df_transac_qtdinad[df_transac_qtdinad['valor_transacoes_12m']<3000].shape

Novamente chegamos a um resultado bastante expressivo de 77,09% dos clientes inadiplentes.

In [ ]:
round(100 * df_qtdinadimplente_cruzado/qtd_inadimplentes_novo, 2)

Para concluirmos iremos verificar qual a porcentagem do total de clientes correspodem esse cruzamento de variáveis.

In [ ]:
#Porcentagem de clientes que transacionaram menos de 60 vezes e sao inadiplentes em relacao ao total de clientes
round(100 * df_qtdinadimplente_cruzado/qtd_total_novo, 2)

Novamente um numero expressivo acima de 12%.

# 7. Conclusões <a name="conclusoes"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topicos)

Baseado em nossas visualizações graficas e os números disponibilizados, podemos afirmar que entre as variáveis apresentadas, as que mais influenciaram na variável `default = 1` isto é ser inadiplente, foram as variáveis `valor_transacoes_12m` e `qtd_transacoes_12m`. Ambas variáveis ou apenas uma delas esta presente em mais de 77% dos clientes, algo bastante expressivo. Recomenda-se buscar novas bases com mais variáveis e parâmetros para se realizar uma analise estatística mais aprofundada e assim entender possiveis causas e criar possiveis soluções de maneira mais assertiva.